## Experiment DL learning with custom embedding and simple Dense layer
Even if the training set is too small to produce a representative word embedding, this notebook illustrates an experiment to have a rough idea on the resulting score with such embedding choice

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from utils import vizu

In [9]:
from sklearn.model_selection import train_test_split

XTrain = pd.read_csv('../../data/staging_data/mispelling_fixed_clean_input_train.csv', sep=',')
YTrain = pd.read_csv('../../data/POSOS/label.csv', sep=';')
XTrain, XTest, YTrain, YTest = train_test_split(XTrain, YTrain, test_size=0.15, random_state=42)

In [10]:
# hyper params
num_classes = 51
patience=20
vocabulary_size = 10000
sequence_length = 30
hidden_dims = 100
embedding_out_dims = 300
dropout_rate=0.2
num_epochs= 100

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(XTrain['question'])

sequences = tokenizer.texts_to_sequences(XTrain['question'])
XEncodedTrain = pad_sequences(sequences, maxlen=sequence_length)

### Build the neural network
The NN is composed of 3 layers:
* custom embeddding layer
* fully connected layer to learn classification with relu activation
* decision layer with sotfmax activation

In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding

import tensorflow as tf
#config = tf.ConfigProto(device_count={"CPU": 32})
config = tf.ConfigProto()
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

model = Sequential()
model.add(Embedding(vocabulary_size , embedding_out_dims, input_length=sequence_length))
model.add(Flatten())
model.add(Dense(hidden_dims, activation='relu'))
model.add(Dropout(rate=dropout_rate))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 300)           3000000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 9000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               900100    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 51)                5151      
Total params: 3,905,251
Trainable params: 3,905,251
Non-trainable params: 0
_________________________________________________________________


### Train the network

In [13]:
YOneHotEncodedTrain = keras.utils.to_categorical(YTrain['intention'])

In [14]:
import keras.utils

call_back_board = keras.callbacks.TensorBoard(
    log_dir='/Users/i051796/git/CES/logs',
    histogram_freq=0,
    batch_size=32,
    write_graph=True,
    write_grads=True,
    write_images=True,
    embeddings_freq=0,
    embeddings_layer_names=None,
    embeddings_metadata=None,
    embeddings_data=None)

# stop criterion to avoid overfitting
call_back_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=patience,
    verbose=0,
    mode='auto',
    baseline=None)

model.fit(
    XEncodedTrain,
    np.array(YOneHotEncodedTrain),
    validation_split=0.1,
    epochs=num_epochs,
    callbacks = [call_back_early_stopping, call_back_board])

Train on 6140 samples, validate on 683 samples
Epoch 1/100
6140/6140 [==============================] - 14s 2ms/step - loss: 2.9948 - acc: 0.2697 - val_loss: 2.5732 - val_acc: 0.3558
Epoch 2/100
6140/6140 [==============================] - 14s 2ms/step - loss: 1.8980 - acc: 0.5318 - val_loss: 1.9863 - val_acc: 0.4919
Epoch 3/100
6140/6140 [==============================] - 14s 2ms/step - loss: 0.9051 - acc: 0.7866 - val_loss: 1.7369 - val_acc: 0.5769
Epoch 4/100
6140/6140 [==============================] - 15s 2ms/step - loss: 0.3734 - acc: 0.9213 - val_loss: 1.7276 - val_acc: 0.5915
Epoch 5/100
6140/6140 [==============================] - 15s 2ms/step - loss: 0.1699 - acc: 0.9691 - val_loss: 1.8171 - val_acc: 0.5900
Epoch 6/100
6140/6140 [==============================] - 15s 2ms/step - loss: 0.0925 - acc: 0.9865 - val_loss: 1.8770 - val_acc: 0.5871
Epoch 7/100
6140/6140 [==============================] - 15s 2ms/step - loss: 0.0671 - acc: 0.9884 - val_loss: 1.9609 - val_acc: 0.5886
E

In [ ]:
%matplotlib inline
vizu.plot_keras_model_learning_curve(model.history)

In [ ]:
import sklearn
from sklearn.metrics import confusion_matrix

testSequences = tokenizer.texts_to_sequences(XTest['question'])
XEncodedTest = pad_sequences(testSequences, maxlen=sequence_length)
    
YTrue = YTest['intention']
YPredicted = model.predict_classes(XEncodedTest)
#YPredicted =list(map(lambda x : getHighProbaLabel(x), YProba))

cnf_matrix = confusion_matrix(YTrue, YPredicted)

print(sklearn.metrics.classification_report(YTrue, YPredicted))

plt.figure(figsize=(20,20))

vizu.plot_confusion_matrix(cnf_matrix, normalize=False, classes = np.unique(YTrain['intention']))

plt.show()


The model is overfitting rapidly beyond 5 epochs but no observed accuracy improvement on validation set (get rapidly stationary accuracy aounrd 62%)

In [ ]:
macroF1Score = sklearn.metrics.f1_score(YTrue, YPredicted, average='macro')
microF1Score = sklearn.metrics.f1_score(YTrue, YPredicted, average='micro')
print ("micro F1 score = {:1.4f} ; macro F1 score = {:1.4f}".format(microF1Score, macroF1Score))